In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import numpy
import cv2
import os
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms,models
import matplotlib.pyplot as plt
import torchvision.transforms as T
from torchvision import transforms
import torch.optim as optim
import copy

#from torchsummary import summary
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
#apply image augmentation using albumentations library.
import albumentations as A
from albumentations.pytorch.transforms  import ToTensor
from numpy import asarray
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import torch.optim as optim
import copy
from sklearn.model_selection import KFold

In [ ]:
train_transforms = transforms.Compose([
                                       transforms.Resize((224,224)),
                                       transforms.ToTensor()])
dataset = datasets.ImageFolder('../input/maskclassification/train/train', transform=train_transforms)
val_rat=0.25
val_size=int(val_rat*len(dataset))
# random_split(range(10), [3, 7], generator=torch.Generator().manual_seed(42))
train_ds, valid_ds = torch.utils.data.random_split(dataset, [(len(dataset) - val_size), val_size], generator=torch.Generator().manual_seed(30))
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
valid_dataloader=torch.utils.data.DataLoader(valid_ds, batch_size=32, shuffle=True)    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")                
 

In [ ]:
def Network():
  
  model = models.resnet18(pretrained=True)
  num_ftrs = model.fc.in_features
  model.fc = nn.Sequential(
    nn.Dropout(0.5),  
    nn.Linear(num_ftrs, 2))
  model.fc = model.fc.cuda() 
  criterion = nn.CrossEntropyLoss()
  model = model.to(device)
  #for param in model.parameters():
    #param.requires_grad = False  
  #for param in model.fc.parameters():
   # param.requires_grad = True  
  lr=0.001
  #optimizer_ft = optim.Adam(model.parameters(), lr=0.001)
  optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
  #This reduces the value of learning rate every 7 steps by a factor of gamma=0.1.
  exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
  return model,optimizer,criterion,exp_lr_scheduler  

In [ ]:
def TrainungFun(model,lr,op,cri,epochs):
  steps = 0
  running_loss = 0
  print_every = 10
  valid_loss_min = np.Inf
  val_lossL = []
  val_acc = []
  train_loss = []
  train_acc = []
  total_step = len(train_dataloader)
  best_model_wts = copy.deepcopy(model.state_dict())
  for i in range(epochs):
     running_loss = 0.0
     correct = 0
     total=0
     current_corrects=0
     with torch.set_grad_enabled(True):
        
        for inputs, labels in train_dataloader:
           steps += 1
           inputs, labels = inputs.to(device), labels.to(device)
           op.zero_grad()
           output = model(inputs)
           loss = cri(output, labels)
           loss.backward()
           op.step()
           running_loss += loss.item()
           _,pred = torch.max(output, dim=1)
           correct += torch.sum(pred==labels).item()
           total += labels.size(0)
           
     val_loss = 0
     accuracy = 0
     model.eval()
     total_t=0
     correct_t=0
     train_acc.append(100 * correct / total)
     train_loss.append(running_loss/total_step)
     with torch.no_grad():
         for inputs_t, target_t in valid_dataloader:
                   
                    inputs_t, target_t = inputs_t.to(device), target_t.to(device)
                      
                    outputs_t = model(inputs_t)
                    batch_loss = cri(outputs_t, target_t)
                    
                    val_loss += batch_loss.item()
                    
                    _,pred_t = torch.max(outputs_t, dim=1)
                    correct_t += torch.sum(pred_t==target_t).item()
                    total_t += target_t.size(0)
     val_acc.append(100 * correct_t/total_t)
     val_lossL.append(val_loss/len(valid_dataloader))
     network_learned = val_loss < valid_loss_min
        
     if   network_learned:
                 best_acc = accuracy
                 best_model_wts = copy.deepcopy(model.state_dict())
                        
                     
           
     print(f"Epoch {i+1}/{epochs},train-loss: {np.mean(train_loss):.3f},train-acc: {(100 * correct/total):.3f}.. "
           f"validation loss: {np.mean(val_lossL):.3f}.. "
           f"validation acc: {(100 * correct_t/total_t):.3f}")  
        
     epoch_loss=running_loss/(len(train_dataloader)+len (valid_dataloader))
  model.load_state_dict(best_model_wts)
  return model
    

In [ ]:
model,optimizer,criterion,learningRate=Network()
model=TrainungFun(model=model,lr=learningRate,op=optimizer,cri=criterion,epochs=15)


In [ ]:
torch.save(model.state_dict(), 'checkpoint.pth')
transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
test_dataset = datasets.ImageFolder('../input/maskclassification/test', transform=transform)
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

import pandas as pd
import os
from pandas import DataFrame           
model.eval()
i=0
output_list=[]
with torch.no_grad():
        for ii,(img, label) in enumerate(test_dataset) :
                    img=torch.tensor(img)
                    img=img.unsqueeze(dim=0)
                    label=torch.tensor(label)
                
                    img, labels = img.to(device), label.to(device)
                    output = model(img)

                    for j in output:
                      im=(os.path.basename(test_dataset.imgs[ii][0]))
                      name=os.path.splitext(im)
                      
                      output_list.append(( im,j.cpu().data.numpy().argmax()))
                      i+=1
                    
                    # you could use any filename. We choose submission here
        df = DataFrame (output_list,columns=['image_name','label'])           
        df.to_csv('submission.csv', index=None)